Code taken from the official BerTopic github/site: https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html \
Code for reducing outliers based on probability: https://github.com/MaartenGr/BERTopic/issues/529 \
Covert output to features: https://stackoverflow.com/questions/73768683/how-to-get-topic-probs-matrix-in-bertopic-modeling

Visual Novel BerTopic

In [ ]:
pip install bertopic


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 111.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 114.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
 

In [ ]:
from bertopic import BERTopic
import pandas
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [ ]:
additional = ["game", "play"]
stop = ENGLISH_STOP_WORDS.union(additional)

In [ ]:
#loading the dataset
df = pandas.read_csv("/content/JRPG_final_Bert.csv")

In [ ]:
df.head()

,app_id,app_name,review_text,review_score,App ID,Genre,Owners,Price,Initial Price,Required Age,Tags
0,107300,Breath of Death VII,This game is a turn based-jrpg with a retro fe...,0,107300,"RPG, Indie","500,000 .. 1,000,000",299,299,0,"JRPG: 305, RPG: 203, Funny: 149, Indie: 148, R..."
1,107300,Breath of Death VII,Breath of Death VII is an old school RPG where...,1,107300,"RPG, Indie","500,000 .. 1,000,000",299,299,0,"JRPG: 305, RPG: 203, Funny: 149, Indie: 148, R..."
2,107300,Breath of Death VII,“The chest screams ‘Bazooka of Ultimate Dest...,0,107300,"RPG, Indie","500,000 .. 1,000,000",299,299,0,"JRPG: 305, RPG: 203, Funny: 149, Indie: 148, R..."
3,107300,Breath of Death VII,"Breath of Death VII is a very short and basic,...",0,107300,"RPG, Indie","500,000 .. 1,000,000",299,299,0,"JRPG: 305, RPG: 203, Funny: 149, Indie: 148, R..."
4,107300,Breath of Death VII,Its sometimes hard for me to get into turn bas...,0,107300,"RPG, Indie","500,000 .. 1,000,000",299,299,0,"JRPG: 305, RPG: 203, Funny: 149, Indie: 148, R..."


In [ ]:
df

,app_id,app_name,review_text,review_score,App ID,Genre,Owners,Price,Initial Price,Required Age,Tags
0,107300,Breath of Death VII,This game is a turn based-jrpg with a retro fe...,0,107300,"RPG, Indie","500,000 .. 1,000,000",299,299,0,"JRPG: 305, RPG: 203, Funny: 149, Indie: 148, R..."
1,107300,Breath of Death VII,Breath of Death VII is an old school RPG where...,1,107300,"RPG, Indie","500,000 .. 1,000,000",299,299,0,"JRPG: 305, RPG: 203, Funny: 149, Indie: 148, R..."
2,107300,Breath of Death VII,“The chest screams ‘Bazooka of Ultimate Dest...,0,107300,"RPG, Indie","500,000 .. 1,000,000",299,299,0,"JRPG: 305, RPG: 203, Funny: 149, Indie: 148, R..."
3,107300,Breath of Death VII,"Breath of Death VII is a very short and basic,...",0,107300,"RPG, Indie","500,000 .. 1,000,000",299,299,0,"JRPG: 305, RPG: 203, Funny: 149, Indie: 148, R..."
4,107300,Breath of Death VII,Its sometimes hard for me to get into turn bas...,0,107300,"RPG, Indie","500,000 .. 1,000,000",299,299,0,"JRPG: 305, RPG: 203, Funny: 149, Indie: 148, R..."
...,...,...,...,...,...,...,...,...,...,...,...
22031,70420,Chantelise,"Flawed but fun, short but sweet, 'Chantelise -...",0,70420,"Action, RPG, Indie","100,000 .. 200,000",999,999,0,"Anime: 137, Action: 112, RPG: 108, Indie: 103,..."
22032,70420,Chantelise,Chantelise is nice if you want to play a game ...,0,70420,"Action, RPG, Indie","100,000 .. 200,000",999,999,0,"Anime: 137, Action: 112, RPG: 108, Indie: 103,..."
22033,70420,Chantelise,Chantelise: The Tale of Two Sisters is a game ...,0,70420,"Action, RPG, Indie","100,000 .. 200,000",999,999,0,"Anime: 137, Action: 112, RPG: 108, Indie: 103,..."
22034,70420,Chantelise,Let's start with I have a 'Very' short amoun...,0,70420,"Action, RPG, Indie","100,000 .. 200,000",999,999,0,"Anime: 137, Action: 112, RPG: 108, Indie: 103,..."


In [ ]:
review = df["review_text"].values

In [ ]:
#preperation
embedding_model = SentenceTransformer("all-mpnet-base-v2")
UMAP_model = UMAP(random_state = 101)
hdbscan_model = HDBSCAN( metric='euclidean', prediction_data = True, min_samples = 3, min_cluster_size = 20)
vectorizer_model = CountVectorizer(min_df = 5, stop_words=list(stop))
ctfidf_model = ClassTfidfTransformer()

In [ ]:
#making topic model
topic_model = BERTopic(embedding_model=embedding_model, umap_model= UMAP_model, hdbscan_model= hdbscan_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model, calculate_probabilities=True, nr_topics=161)

In [ ]:
topics, probs = topic_model.fit_transform(review)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,3931,-1_like_just_story_good
1,0,3722,0_fantasy_final_ff_xiii
2,1,790,1_port_pc_version_fps
3,2,631,2_mmo_class_classes_mmos
4,3,580,3_neptunia_nep_hyperdimension_birth
...,...,...,...
156,155,20,155_free_warning_save_items
157,156,20,156_legionwood_throne_amber_maker
158,157,20,157_tales_zestiria_bandai_port
159,158,20,158_daughter_princess_dos_refine


In [ ]:
topic_model.get_topic(9)

[('port', 0.014372341518881922),
 ('controller', 0.013990864109819772),
 ('keyboard', 0.010257163150466165),
 ('mouse', 0.009307981712736035),
 ('pc', 0.009267390901420569),
 ('version', 0.008587725383251707),
 ('controls', 0.007156488990817218),
 ('psp', 0.007054212093083681),
 ('issues', 0.006549964921115073),
 ('camera', 0.006477613186778568)]

In [ ]:
# Use the "c-TF-IDF" strategy with a threshold
new_topics = topic_model.reduce_outliers(review, topics , strategy="c-tf-idf")

In [ ]:
topic_model.update_topics(review, topics=new_topics, vectorizer_model=vectorizer_model)

In [ ]:
topic_model.get_topic_info().sort_values("Count", ascending = False).iloc[[9]]

,Topic,Count,Name
12,12,461,12_maker_rpgmaker_rpg_games


In [ ]:
topic_model.get_topic(12)

[('maker', 0.023993058107641758),
 ('rpgmaker', 0.019953206593966027),
 ('rpg', 0.01407030719734584),
 ('games', 0.0065334197779622905),
 ('story', 0.005837003980698763),
 ('good', 0.00569238971650137),
 ('like', 0.005112618234767279),
 ('really', 0.004911459602430678),
 ('don', 0.004608645587801738),
 ('did', 0.004549954621615761)]

In [ ]:
#getting output features
output_features = pandas.DataFrame(probs)
output_features["sentiment"] = df["review_score"]


In [ ]:
output_features

In [ ]:
#saving model and features
topic_model.save("JRPG_model_Bert")



In [ ]:
output_features.to_csv("features_JRPG_Bert_reduction.csv")

In [ ]:
from google.colab import files
files.download('JRPG_model_Bert')